In [83]:
import logging
import os
from dataclasses import dataclass, field
from typing import Dict, List, Optional

import numpy as np
import torch

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from tqdm import tqdm
from transformers.trainer_utils import is_main_process
from utils_hans import HansDataset, InputFeatures, hans_processors, hans_tasks_num_labels


logger = logging.getLogger(__name__)

In [85]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")

In [86]:
dataset = HansDataset(
            data_dir='/home/nlp/data/glue_data/hans',
            tokenizer=tokenizer,
            task='hans',
            max_seq_length=96,
            evaluate=True
        )
from datasets import load_dataset
ds = load_dataset("hans", split="validation")
gt = []
for val in ds:
    gt.append(val['label'])
def put_to_cuda(example):
    val = {}
    val['input_ids'] = example.input_ids.unsqueeze(0).cuda()
    val['attention_mask'] = example.attention_mask.cuda()
    val['token_type_ids'] = example.token_type_ids.cuda()
    return val

Reusing dataset hans (/home/nlp/.cache/huggingface/datasets/hans/plain_text/1.0.0/ccd7261ed79f0e126a724d1c0b6d94d8e029b9ec309c8e6c5dbece3538ab7f3b)


In [105]:
model = AutoModelForSequenceClassification.from_pretrained("/home/nlp/experiments/again_bert_medium_59")
model.cuda();

In [106]:
model_outputs = []
for example in tqdm(dataset):
    cuda_input = put_to_cuda(example)
    output = model(**cuda_input)
    model_label = torch.argmax(output.logits.squeeze(0)).item()
    model_outputs.append(model_label)

100%|██████████| 30000/30000 [03:40<00:00, 136.19it/s]


In [107]:
for idx, mo in enumerate(model_outputs):
    if mo == 2:
        model_outputs[idx] = 1
sum(np.array(model_outputs) == np.array(gt))/30000

0.5122333333333333

In [108]:
val_1 = 50.65
val_2 = 51.21
val_3 = 51.2

np.average([val_1, val_2, val_3]), np.std([val_1, val_2, val_3])

(51.02, 0.26166135875720636)